# DATA 512: A1 Data Curation Assignment
By: Megan Nalani Chun

## Step 1: Gathering the data <br>
Wikipedia traffic from Jan 1, 2008 - August 30, 2020 <br>
- Legacy Pagecounts API provides desktop and mobile traffic data from Dec. 2007 - July 2016 <br>
- Pageviews API provides desktop, mobile web, and mobile app traffic data from July 2015 - last month. 


First, import the json and requests libraries to call the Pagecounts and Pageviews APIs and save the output in json format.

In [1]:
import json
import requests

Second, set the location of the endpoints and header information

In [2]:
endpoint_legacy = 'https://wikimedia.org/api/rest_v1/metrics/legacy/pagecounts/aggregate/{project}/{access-site}/{granularity}/{start}/{end}'

endpoint_pageviews = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/aggregate/{project}/{access}/{agent}/{granularity}/{start}/{end}'

headers = {
    'User-Agent': 'https://github.com/NalaniKai/',
    'From': 'nalani23@uw.edu'
}

Third, define a function to call the APIs taking in the endpoint and parameters (defined below)

In [3]:
def api_call(endpoint,parameters):
    call = requests.get(endpoint.format(**parameters), headers=headers)
    response = call.json()
    
    return response

Fourth, define a function to make the api_call() with the correct parameters and save the output to json file in format apiname_accesstype_startyearmonth_endyearmonth.json

In [4]:
def get_data(api, access_dict, params, endpoint):
    start = 0 #index of start date
    end = 1   #index of end data
    year_month = 6 #size of YYYYMM
    for access_type, start_end_dates in access_dict.items():
        params["access-site"] = access_type
        data = api_call(endpoint, params)
        with open(api + "_" + access_type + "_" + start_end_dates[start][:year_month] + "_" + start_end_dates[end][:year_month] + ".json", 'w') as f:
            json.dump(data, f)

Fifth, define the parameters for the legacy page count API and call the get_data function. This will pull the data and save it.

In [37]:
api = "pagecounts"
access_type_legacy = {"desktop-site": ["2008010100", "2016070100"], 
                     "mobile-site": ["2014100100", "2016070100"]}
# see: https://wikimedia.org/api/rest_v1/#!/Legacy_data/get_metrics_legacy_pagecounts_aggregate_project_access_site_granularity_start_end
params_legacy = {"project" : "en.wikipedia.org",
                 #"access-site" : "desktop-site",
                 "granularity" : "monthly",
                 "start" : "2008010100",
                # for end use 1st day of month following final month of data
                 "end" : "2016080100"
                    }
get_data(api, access_type_legacy, params_legacy, endpoint_legacy)

Sixth, define the parameters for the page views API and call the get_data function. This will pull the data and save it.

In [7]:
api = "pageviews"
start_end_dates = ["2015070100", "2020080100"]
access_type_pageviews = {"desktop": start_end_dates, 
                        "mobile-app": start_end_dates, 
                        "mobile-web": start_end_dates }
# see: https://wikimedia.org/api/rest_v1/#!/Pageviews_data/get_metrics_pageviews_aggregate_project_access_agent_granularity_start_end
params_pageviews = {"project" : "en.wikipedia.org",
                    "access" : "mobile-web",
                    "agent" : "user",
                    "granularity" : "monthly",
                    "start" : "2008010100",
                    # for end use 1st day of month following final month of data
                    "end" : '2020090100'
                        }
get_data(api, access_type_pageviews, params_pageviews, endpoint_pageviews)                       

## Step 2: Processing the data

First, create a function to read in all data files and extract the list of records from items.

In [8]:
def read_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)["items"]

Second, use the read_json function to get a list of records for each file.

In [38]:
pageviews_mobile_web = read_json("pageviews_mobile-web_201507_202008.json")
pageviews_mobile_app = read_json("pageviews_mobile-app_201507_202008.json")
pageviews_desktop = read_json("pageviews_desktop_201507_202008.json")
pagecounts_mobile = read_json("pagecounts_mobile-site_201410_201607.json")
pagecounts_desktop = read_json("pagecounts_desktop-site_200801_201607.json")

Third, create a total mobile traffic count for each month using the mobile-app and mobile-web data from the pageviews API. The list of [timestamp, view_count] pairs data structure will enable easy transformation into a dataframe.

In [39]:
pageviews_mobile = [[r1["timestamp"], r0["views"] + r1["views"]] for r0 in pageviews_mobile_web for r1 in pageviews_mobile_app if r0["timestamp"] == r1["timestamp"]]

Fourth, get the timestamps and values for the desktop pageviews, desktop pagecounts, and mobile pagecounts. The list of [timestamp, view_count] pairs data structure will enable easy transformation into a dataframe.

In [40]:
pageviews_desktop = [[record["timestamp"], record["views"]] for record in pageviews_desktop]
pagecounts_desktop = [[record["timestamp"], record["count"]] for record in pagecounts_desktop]
pagecounts_mobile = [[record["timestamp"], record["count"]] for record in pagecounts_mobile]

Fifth, import pandas library and transform data into dataframes.

In [41]:
import pandas as pd 
pageview_desktop_views = pd.DataFrame(pageviews_desktop, columns=["timestamp", "pageview_desktop_views"])
pageview_mobile_views = pd.DataFrame(pageviews_mobile, columns=["timestamp", "pageview_mobile_views"])
pagecounts_desktop = pd.DataFrame(pagecounts_desktop, columns=["timestamp", "pagecount_desktop_views"])
pagecounts_mobile = pd.DataFrame(pagecounts_mobile, columns=["timestamp", "pagecount_mobile_views"])

Sixth, join page view dataframes and calculate total for all views.

In [42]:
df_pageviews = pd.merge(pageview_desktop_views, pageview_mobile_views, how="outer", on="timestamp")
df_pageviews["pageview_all_views"] = df_pageviews["pageview_desktop_views"] + df_pageviews["pageview_mobile_views"]
df_pageviews.head()

,timestamp,pageview_desktop_views,pageview_mobile_views,pageview_all_views
0,2015070100,3179131148,6358262296,9537393444
1,2015080100,3192663889,6385327778,9577991667
2,2015090100,3073981649,6147963298,9221944947
3,2015100100,3173975355,6347950710,9521926065
4,2015110100,3142247145,6284494290,9426741435


Seventh, join page count dataframes, fill in NaN values with 0, and calculate total for all counts.

In [43]:
df_pagecounts = pd.merge(pagecounts_desktop, pagecounts_mobile, how="outer", on="timestamp")
df_pagecounts["pagecount_mobile_views"] = df_pagecounts["pagecount_mobile_views"].fillna(0)
df_pagecounts["pagecount_all_views"] = df_pagecounts["pagecount_desktop_views"] + df_pagecounts["pagecount_mobile_views"]
df_pagecounts.head()

,timestamp,pagecount_desktop_views,pagecount_mobile_views,pagecount_all_views
0,2008010100,4930902570,0.0,4.930903e+09
1,2008020100,4818393763,0.0,4.818394e+09
2,2008030100,4955405809,0.0,4.955406e+09
3,2008040100,5159162183,0.0,5.159162e+09
4,2008050100,5584691092,0.0,5.584691e+09


Eighth, join page count and page view dataframes into one table. Filling in missing values with 0.

In [44]:
df = pd.merge(df_pagecounts, df_pageviews, how="outer", on="timestamp")
df = df.fillna(0)
df.head()

,timestamp,pagecount_desktop_views,pagecount_mobile_views,pagecount_all_views,pageview_desktop_views,pageview_mobile_views,pageview_all_views
0,2008010100,4.930903e+09,0.0,4.930903e+09,0.0,0.0,0.0
1,2008020100,4.818394e+09,0.0,4.818394e+09,0.0,0.0,0.0
2,2008030100,4.955406e+09,0.0,4.955406e+09,0.0,0.0,0.0
3,2008040100,5.159162e+09,0.0,5.159162e+09,0.0,0.0,0.0
4,2008050100,5.584691e+09,0.0,5.584691e+09,0.0,0.0,0.0


Ninth, separate the timestamp into the year and month in YYYY and MM format for all the data. Remove the timestamp column.

In [45]:
df["year"] = df["timestamp"].apply(lambda date: date[:4])
df["month"] = df["timestamp"].apply(lambda date: date[4:6])
df.drop("timestamp", axis=1, inplace=True)
df.head()

,pagecount_desktop_views,pagecount_mobile_views,pagecount_all_views,pageview_desktop_views,pageview_mobile_views,pageview_all_views,year,month
0,4.930903e+09,0.0,4.930903e+09,0.0,0.0,0.0,2008,01
1,4.818394e+09,0.0,4.818394e+09,0.0,0.0,0.0,2008,02
2,4.955406e+09,0.0,4.955406e+09,0.0,0.0,0.0,2008,03
3,5.159162e+09,0.0,5.159162e+09,0.0,0.0,0.0,2008,04
4,5.584691e+09,0.0,5.584691e+09,0.0,0.0,0.0,2008,05


Tenth, save processed data to csv file.

In [46]:
df.to_csv("en-wikipedia_traffic_200801-202008.csv", index=False)

## Step 3: Analyze the data

In [47]:
import matplotlib.pyplot as plt 
from datetime import date
import numpy as np
df.replace(0, np.nan, inplace=True)
date = df.apply(lambda r: date(int(r["year"]), int(r["month"]), 1), axis=1) 
pc_mobile = df["pagecount_mobile_views"] / 1e6
pv_mobile = df["pageview_mobile_views"] / 1e6
pc_desktop = df["pagecount_desktop_views"] / 1e6
pv_desktop = df["pageview_desktop_views"] / 1e6
pv_total = df["pageview_all_views"] / 1e6
pc_total = df["pagecount_all_views"] / 1e6
df.tail()

,pagecount_desktop_views,pagecount_mobile_views,pagecount_all_views,pageview_desktop_views,pageview_mobile_views,pageview_all_views,year,month
147,NaN,NaN,NaN,5.332822e+09,1.066564e+10,1.599847e+10,2020,04
148,NaN,NaN,NaN,5.089055e+09,1.017811e+10,1.526717e+10,2020,05
149,NaN,NaN,NaN,4.552042e+09,9.104084e+09,1.365613e+10,2020,06
150,NaN,NaN,NaN,4.675167e+09,9.350333e+09,1.402550e+10,2020,07
151,NaN,NaN,NaN,4.647875e+09,9.295750e+09,1.394363e+10,2020,08


In [36]:
test = df[df["year"] == "2016"]
test[test["month"]=="07"]

,pagecount_desktop_views,pagecount_mobile_views,pagecount_all_views,pageview_desktop_views,pageview_mobile_views,pageview_all_views,year,month
102,5.363966e+09,3.500661e+09,8.864628e+09,3.395175e+09,6.790350e+09,1.018553e+10,2016,07


In [48]:
from matplotlib.dates import YearLocator
fig, ax1 = plt.subplots(figsize=(18,6))
ax1.plot(date, pc_desktop, label="main site", color="green", ls="--")
ax1.plot(date, pv_desktop, label="_Hidden label", color="green", ls="-")
ax1.plot(date, pc_mobile, label="mobile site", color="blue", ls="--")
ax1.plot(date, pv_mobile, label="_Hidden label", color="blue", ls="-")
ax1.plot(date, pc_total, label="total", color="black", ls="--")
ax1.plot(date, pv_total, label="_Hidden label", color="black", ls="-")
ax1.xaxis.set_major_locator(YearLocator())
plt.xlabel("Date")
plt.ylabel("View Count")
plt.ylim(ymin=0)
plt.title("Page Views on English Wikipedia (x 1,000,000)")
plt.grid(True)
plt.legend(loc="upper left")
plt.show()